In [1]:
# Sempre rode isso para começar o Notebook
# De modo resumido, isso faz com que o notebook possa acessar o Django

import sys
sys.path.append("../../")

import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serase_projeto.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
# Digite seu código aqui!

In [3]:
# Testando values_list no PadraoMovimentacao

from serase_app.models import PadraoMovimentacao
from django.db.models import F

lista = PadraoMovimentacao.objects.values("id", "descricao", "periodo", "dia_cobranca", "data_inicio", "data_fim", valor_padrao=F("valor"), categoria=F("cod_categoria__nome"), tipo=F("receita_despesa"))
lista = list(lista)
lista

[{'id': 1,
  'descricao': 'Spotify',
  'periodo': 1,
  'dia_cobranca': 1,
  'data_inicio': datetime.date(2018, 4, 18),
  'data_fim': None,
  'valor_padrao': Decimal('-26.90'),
  'categoria': 'Lazer',
  'tipo': 'despesa'},
 {'id': 2,
  'descricao': 'Bolsa IC',
  'periodo': 1,
  'dia_cobranca': 6,
  'data_inicio': datetime.date(2020, 7, 8),
  'data_fim': datetime.date(2021, 7, 8),
  'valor_padrao': Decimal('300.00'),
  'categoria': 'Sálario',
  'tipo': 'receita'},
 {'id': 3,
  'descricao': 'Conta de luz',
  'periodo': 1,
  'dia_cobranca': 6,
  'data_inicio': None,
  'data_fim': None,
  'valor_padrao': None,
  'categoria': 'Habitação',
  'tipo': 'despesa'},
 {'id': 4,
  'descricao': 'Internet',
  'periodo': 1,
  'dia_cobranca': 5,
  'data_inicio': datetime.date(2020, 2, 4),
  'data_fim': None,
  'valor_padrao': Decimal('-59.90'),
  'categoria': 'Habitação',
  'tipo': 'despesa'},
 {'id': 5,
  'descricao': 'Sálario',
  'periodo': 1,
  'dia_cobranca': 5,
  'data_inicio': None,
  'data_fim': 

In [4]:
# Testando o choices do PadraoMovimentacao

#from serase_app.models import PadraoMovimentacao
from django.db.models import F, Case, When, CharField, Value

# > Consulta

# Pego de https://stackoverflow.com/a/60158573
class WithChoices(Case):
    def __init__(self, model, field, condition=None, then=None, **lookups):
        choices = dict(model._meta.get_field(field).flatchoices)
        whens = [When(**{field: k, 'then': Value(v)}) for k, v in choices.items()]
        return super().__init__(*whens, output_field=CharField())


PadraoMovimentacao.objects.values()

<QuerySet [{'id': 1, 'receita_despesa': 'despesa', 'descricao': 'Spotify', 'periodo': 1, 'valor': Decimal('-26.90'), 'dia_cobranca': 1, 'data_inicio': datetime.date(2018, 4, 18), 'data_fim': None, 'cod_usuario_id': 1, 'cod_categoria_id': 4}, {'id': 2, 'receita_despesa': 'receita', 'descricao': 'Bolsa IC', 'periodo': 1, 'valor': Decimal('300.00'), 'dia_cobranca': 6, 'data_inicio': datetime.date(2020, 7, 8), 'data_fim': datetime.date(2021, 7, 8), 'cod_usuario_id': 1, 'cod_categoria_id': 9}, {'id': 3, 'receita_despesa': 'despesa', 'descricao': 'Conta de luz', 'periodo': 1, 'valor': None, 'dia_cobranca': 6, 'data_inicio': None, 'data_fim': None, 'cod_usuario_id': 1, 'cod_categoria_id': 10}, {'id': 4, 'receita_despesa': 'despesa', 'descricao': 'Internet', 'periodo': 1, 'valor': Decimal('-59.90'), 'dia_cobranca': 5, 'data_inicio': datetime.date(2020, 2, 4), 'data_fim': None, 'cod_usuario_id': 1, 'cod_categoria_id': 10}, {'id': 5, 'receita_despesa': 'receita', 'descricao': 'Sálario', 'periodo

In [5]:
# Testando movimentacoes

from serase_app.models import *

In [6]:
valores = Movimentacao.objects.filter(valor_pago__isnull=False).values_list("descricao", "data_lancamento").order_by("-data_lancamento")

for nom, dat in valores:
    print(nom + " " + str(dat))

Mate coro e sanduba 2020-11-01
bebida Halloween 2020-10-30
Make pro Halloween 2020-10-30
Dinheiro do Spotify 2020-10-26
Quebrando a quarentena 2020-10-17
Compras 2020-10-10
Bolsa ic 2020-10-08
Conta de luz 2020-10-08
Internet 2020-10-08
Spotify 2020-10-08
Sálario 2020-10-07
Churros 2020-10-03
Milkshake McDonnadls 2020-09-15
Compras 2020-09-12
Bolsa ic 2020-09-08
Conta de luz 2020-09-08
Internet 2020-09-08
Internet 2020-09-08
Spotify 2020-09-08
Sálario 2020-09-07
Viola 2020-09-02
Bolsa ic 2020-08-10
Conta de luz 2020-08-10
Internet 2020-08-10
Spotify 2020-08-10
Sálario 2020-08-07
Adoção do Cumpa, o Cacto 2020-07-16
Outer wilds 2020-07-10


datetime.datetime(2020, 11, 20, 15, 29, 33, 750615)

In [28]:
# Teste datetime: objeto data apartir de string e filtrar por data

from datetime import datetime

data = datetime.strptime("2020-10-26", '%Y-%m-%d').date()
Movimentacao.objects.filter(data_lancamento__gte=data)

<QuerySet [<Movimentacao: Mate coro e sanduba>, <Movimentacao: bebida Halloween>, <Movimentacao: Make pro Halloween>, <Movimentacao: Dinheiro do Spotify>]>

In [5]:
a = Movimentacao.objects.filter(descricao="Churros").values()
list(a)[0]

{'id': 13,
 'descricao': 'Churros',
 'valor_esperado': None,
 'valor_pago': Decimal('-5.00'),
 'data_geracao': None,
 'data_lancamento': datetime.date(2020, 10, 3),
 'cod_usuario_id': 1,
 'cod_categoria_id': 5,
 'cod_padrao_id': None}

In [19]:
from datetime import datetime

mes_ano = datetime.strptime("2020-10", '%Y-%m')

a = Movimentacao.objects.filter(data_lancamento__month=mes_ano.month, data_lancamento__year=mes_ano.year).order_by("-data_lancamento")
for q in a:
    print(q.descricao, q.data_lancamento)

bebida Halloween 2020-10-30
Make pro Halloween 2020-10-30
Dinheiro do Spotify 2020-10-26
Quebrando a quarentena 2020-10-17
Compras 2020-10-10
Bolsa ic 2020-10-08
Conta de luz 2020-10-08
Internet 2020-10-08
Spotify 2020-10-08
Sálario 2020-10-07
Churros 2020-10-03


In [21]:
datetime.today().year

2020

In [27]:
from django.db.models import Sum

mes_ano = datetime.strptime("2020-10", '%Y-%m')
query_movimentacao = Movimentacao.objects.filter(valor_pago__isnull=False)
query_movimentacao = query_movimentacao.filter(data_lancamento__year=mes_ano.year, data_lancamento__month=mes_ano.month)
query_movimentacao.aggregate(Sum("valor_pago"))["valor_pago__sum"]

Decimal('2126.5')

In [4]:
from datetime import datetime, timedelta

def mes_passado(data):
    dia = data.day
    data = data.replace(day=1) - timedelta(days=1)
    data = data.replace(day=dia)
    return data

mes_ano = datetime.strptime("2020-01", '%Y-%m')
mes_passado(mes_ano), mes_ano

(datetime.datetime(2019, 12, 1, 0, 0), datetime.datetime(2020, 1, 1, 0, 0))

In [3]:
from serase_relatorio.utils import calcula_periodo, calcula_periodo_anterior
from serase_app.models import *

from datetime import datetime, timedelta
from django.db.models import Q, F, Sum, Case, When, DecimalField, Value
from django.db.models.functions import Cast, Coalesce


usuario = User.objects.get(username="jv_eumsmo")

hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
periodo = "semanal"

data_inicio, data_fim = calcula_periodo(periodo,hoje)
data_inicio_passado, data_fim_passado = calcula_periodo_anterior(periodo, hoje)

query = Movimentacao.objects.filter(cod_usuario=usuario, valor_pago__lt=0) # Seleciona despesas do usuario
query = query.values('cod_categoria')

# Seleciona despesas do periodo atual
qperiodo_atual = query.filter(data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)
    # Soma todas as despesas de uma categoria em 'despesa_total' e ordena em ordem decrescente
qperiodo_atual = qperiodo_atual.annotate(despesa_total=Sum('valor_pago')).order_by("despesa_total")

print(data_inicio, data_fim)
print(data_inicio_passado, data_fim_passado)

query.filter(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim).values('cod_categoria').annotate(
    despesa_total=Case(
        When(data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim, then=Sum('valor_pago')),
        default=Value(0.0),
        output_field=DecimalField()
    ) - Case(
        When(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim_passado, then=Sum('valor_pago')),
        default=Value(0.0),
        output_field=DecimalField()
    )
).aggregate(a=Sum('despesa_total'))

query.filter(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim).annotate(
    despesa_total=Coalesce(Sum('valor_pago', filter=Q(data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)), 0.0) - Coalesce(Sum('valor_pago', filter=Q(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim_passado)),0.0)
)

# Seleciona despesas entre o começo do ultimo periodo e o final do periodo atual
qrelacao_ultimo = query.filter(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim)
    
    #
qrelacao_ultimo = qrelacao_ultimo.annotate(
    despesa_total= Coalesce(Sum('valor_pago', filter=Q(data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)), 0.0) 
    - Coalesce(Sum('valor_pago', filter=Q(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim_passado)),0.0)
)

qrelacao_ultimo.order_by('despesa_total').last()
#qperiodo_antes.union(qperiodo_atual.annotate(despesa_total=-1*F('despesa_total'))).values('cod_categoria','despesa_total').annotate(diferenca=Sum('despesa_total'))
#qperiodo_atual, qperiodo_antes

ModuleNotFoundError: No module named 'serase_relatorio'

In [4]:
qperiodo_atual

NameError: name 'qperiodo_atual' is not defined

In [5]:
from serase_relatorio.utils import calcula_periodo, calcula_periodo_anterior
from serase_app.models import *

from datetime import datetime, timedelta
from django.db.models import Q, F, Sum,  DecimalField, Value, Count
from django.db.models.functions import Cast, Coalesce


hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
periodo = "ano"
data_inicio, data_fim = calcula_periodo(periodo, hoje)

query = Movimentacao.objects.filter(cod_usuario=usuario, data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)
query = query.filter(valor_pago__isnull=False)

total_despesas = query.count()

query = query.values(nome=F("cod_categoria__nome")).annotate(porcentagem=Cast(100.0 * Count("cod_categoria")/float(total_despesas), DecimalField(decimal_places=2)))
list(query)

ModuleNotFoundError: No module named 'serase_relatorio'

In [6]:
from serase_relatorio.utils import calcula_periodo, calcula_periodo_anterior
from serase_app.models import *

from datetime import datetime, timedelta
from django.db.models import Q, F, Case, When, Value, Count, CharField, DecimalField
from django.db.models.functions import Cast, Coalesce, Extract


hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
periodo = "anual"
data_inicio, data_fim = calcula_periodo(periodo, hoje)

query = Movimentacao.objects.filter(cod_usuario=usuario, data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)
query = query.filter(cod_padrao__isnull=False, valor_pago__isnull=False, valor_pago__lt=0)

query = query.annotate(tipo=Case(
    When(cod_padrao__valor__isnull=True, then=Value("Variáveis")),
    default=Value("Fixas"),
    output_field=CharField()
))

total = query.count()

query.values("tipo").annotate(porcentagem=Cast(100.0 * Count("tipo")/float(total), DecimalField(decimal_places=2)))

ModuleNotFoundError: No module named 'serase_relatorio'

In [14]:
from serase_relatorio.utils import calcula_periodo, calcula_periodo_anterior
from serase_app.models import *

from datetime import datetime, timedelta
from django.db.models import Q, F, Case, When, Value, Count, CharField, Sum
from django.db.models.functions import Cast, Coalesce, Extract

usuario = User.objects.get(username="jv_eumsmo")

hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
periodo = "anual"
data_inicio, data_fim = calcula_periodo(periodo, hoje)

query = Movimentacao.objects.filter(cod_usuario=usuario, data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)
query = query.filter(cod_padrao__isnull=False, valor_pago__isnull=False, valor_pago__lt=0, cod_padrao__valor__isnull=True)


query = query.annotate(mes=Extract("data_lancamento", "month")).values("mes", nome=F("cod_padrao__descricao")).annotate(valor=Sum("valor_pago"))

resultado = dict()
query = list(query)

for obj in query:
    valor = {"mes": obj["mes"], "valor": round(float(obj["valor"]), 2)}
    
    if obj["nome"] not in resultado:
        resultado[obj["nome"]] = list()
    resultado[obj["nome"]].append(valor)

resultado

{'Conta de luz': [{'mes': 8, 'valor': -102.8},
  {'mes': 9, 'valor': -86.5},
  {'mes': 10, 'valor': -95.25}],
 'Padrao Teste': [{'mes': 10, 'valor': -15.0}]}

In [16]:
hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
periodo = "anual"

data_inicio, data_fim = calcula_periodo(periodo,hoje)
data_inicio_passado, data_fim_passado = calcula_periodo_anterior(periodo, hoje)

data_inicio, data_fim, data_inicio_passado, data_fim_passado

(datetime.datetime(2020, 1, 1, 0, 0),
 datetime.datetime(2020, 12, 31, 0, 0),
 datetime.datetime(2019, 1, 1, 0, 0),
 datetime.datetime(2019, 12, 31, 0, 0))

In [18]:
Categoria.objects.get(id=8)

<Categoria: Categoria object (8)>

In [28]:
from django.db.models import Q, F, Case, When, Value, Count, CharField, Sum
from django.db.models.functions import Cast, Coalesce, Extract

Movimentacao.objects.filter(data_lancamento__isnull=False).annotate(mes=Extract("data_lancamento", "month"), ano=Extract("data_lancamento", "year")).values("mes","ano").distinct()

<QuerySet [{'mes': 11, 'ano': 2020}, {'mes': 10, 'ano': 2020}, {'mes': 9, 'ano': 2020}, {'mes': 8, 'ano': 2020}, {'mes': 7, 'ano': 2020}]>